In [ ]:
import os
import time
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from numpy.linalg import svd, norm
from memory_profiler import memory_usage

# --- File paths ---
base_path = r'G:\My Drive\NUS\NUS Y6S1\ME5311\PROJECT_2420_ME5311'
slp_path = os.path.join(base_path, 'slp.nc')
t2m_path = os.path.join(base_path, 't2m.nc')

# --- Load datasets ---
ds_slp = xr.open_dataset(slp_path)
ds_t2m = xr.open_dataset(t2m_path)

slp = ds_slp['msl'].values
t2m = ds_t2m['t2m'].values
timestamps = ds_slp['time'].values
lats = ds_slp['latitude'].values
longs = ds_slp['longitude'].values

# --- Reshape and center SLP ---
n_time, n_lat, n_lon = slp.shape
A_slp = slp.reshape(n_time, -1).T
A_mean_slp = A_slp.mean(axis=1, keepdims=True)
A_centered_slp = A_slp - A_mean_slp

In [ ]:
# --- Runtime and memory tracking wrapper ---
def perform_svd_with_monitoring(A):
    def svd_task():
        global U_slp, S_slp, VT_slp
        U_slp, S_slp, VT_slp = svd(A, full_matrices=True)

    start = time.time()
    mem_usage = memory_usage(svd_task, max_usage=True)
    elapsed = time.time() - start
    return elapsed, mem_usage

# --- Perform full classical SVD and monitor performance ---
print("Performing full classical SVD on SLP...")
elapsed_slp, peak_mem_slp = perform_svd_with_monitoring(A_centered_slp)




In [ ]:
# --- Accuracy (Reconstruction Error) with full S matrix ---
S_full = np.zeros((U_slp.shape[1], VT_slp.shape[0]))
np.fill_diagonal(S_full, S_slp)
A_reconstructed = U_slp @ S_full @ VT_slp + A_mean_slp
reconstruction_error = norm(A_slp - A_reconstructed) / norm(A_slp)

# --- Noise Robustness Test ---
np.random.seed(0)
noise = np.random.normal(scale=0.01, size=A_centered_slp.shape)
A_noisy = A_centered_slp + noise
U_noisy, S_noisy, VT_noisy = svd(A_noisy, full_matrices=True)
S_noisy_full = np.zeros((U_noisy.shape[1], VT_noisy.shape[0]))
np.fill_diagonal(S_noisy_full, S_noisy)
A_reconstructed_noisy = U_noisy @ S_noisy_full @ VT_noisy + A_mean_slp
noise_error = norm(A_slp - A_reconstructed_noisy) / norm(A_slp)

# --- Report results ---
print("\n===== Full Classical SVD Results for SLP =====")
print(f"A shape: {A_slp.shape}")
print(f"U shape: {U_slp.shape}, S shape: {S_slp.shape}, VT shape: {VT_slp.shape}")
print(f"Runtime: {elapsed_slp:.2f} seconds")
print(f"Peak memory usage: {peak_mem_slp:.2f} MiB")
print(f"Reconstruction error (Frobenius norm): {reconstruction_error:.6e}")
print(f"Noise robustness (error with Gaussian noise): {noise_error:.6e}")



===== Full Classical SVD Results for SLP =====
A shape: (16261, 16071)
U shape: (16261, 16261), S shape: (16071,), VT shape: (16071, 16071)
Runtime: 1137.30 seconds
Peak memory usage: 19839.78 MiB
Reconstruction error (Frobenius norm): 2.146784e-10
Noise robustness (error with Gaussian noise): 1.031716e-07

In [ ]:
# --- Report results ---
print("\n===== Full Classical SVD Results for SLP =====")
print(f"Runtime: {elapsed_slp:.2f} seconds")
print(f"Peak memory usage: {peak_mem_slp:.2f} MiB")
print(f"Reconstruction error (Frobenius norm): {reconstruction_error:.6e}")
print(f"Noise robustness (error with Gaussian noise): {noise_error:.6e}")

# --- Optional: Save results to file for later comparison ---
results = {
    "method": "Full Classical SVD",
    "runtime": elapsed_slp,
    "memory_usage": peak_mem_slp,
    "reconstruction_error": reconstruction_error,
    "noise_robustness": noise_error,
    "top_singular_values": S_slp[:10].tolist()  # Save first 10 singular values
}

# Save as JSON (optional)
import json
with open("full_svd_results.json", "w") as f:
    json.dump(results, f, indent=4)

# --- Optional: Plot singular value decay ---
plt.figure(figsize=(10, 6))
plt.semilogy(range(1, 101), S_slp[:100], 'o-')
plt.title('Singular Value Decay (Top 100)')
plt.xlabel('Index')
plt.ylabel('Singular Value (log scale)')
plt.grid(True)
plt.savefig('full_svd_singular_values.png', dpi=300)
plt.show()